In [ ]:
import streamlit as st

import geemap, ee
import geopandas as gpd
from shapely.geometry import Polygon

st.set_page_config(layout="wide")
@st.cache_data
def ee_authenticate(token_name="EARTHENGINE_TOKEN"):
    geemap.ee_initialize(token_name=token_name)

def getNDVI(image):    
    # Normalized difference vegetation index (NDVI)
    ndvi = image.normalizedDifference(['B8','B4']).rename("NDVI")
    image = image.addBands(ndvi)

    return(image)

def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())


st.title("NDVI Map Canterbury Region")
ee_authenticate(token_name="EARTHENGINE_TOKEN")

aoi = ee.FeatureCollection("FAO/GAUL/2015/level1").filter(ee.Filter.eq('ADM1_NAME','Canterbury')).geometry()

NDVI_data = ee.ImageCollection('COPERNICUS/S2_SR').filterDate("2023-01-01","2023-01-31").filterBounds(aoi) \
    .map(getNDVI).map(addDate).max()

color = ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
               '74A901', '66A000', '529400', '3E8601', '207401', '056201',
               '004C00', '023B01', '012E01', '011D01', '011301']
pallete = {"min":0, "max":1, 'palette':color}
# initialize our map
map1 = geemap.Map()
map1.centerObject(aoi, 7)
map1.addLayer(NDVI_data.clip(aoi).select('NDVI'), pallete, "NDVI-Canterbury")

map1.addLayerControl()
map1

# map1.to_streamlit(height=700)


In [ ]:
ee_authenticate(token_name="EARTHENGINE_TOKEN")

aoi = ee.FeatureCollection("FAO/GAUL/2015/level1").filter(ee.Filter.eq('ADM1_NAME','Canterbury')).geometry()

coordcan = aoi.coordinates().getInfo()
# polygons = [Polygon(polygon_coords) for polygon_coords in coordcan]

In [10]:
import geemap #.foliumap as geemap
import ee
Map = geemap.Map()

# roi = Map.draw_last_feature()

roi = ee.FeatureCollection("FAO/GAUL/2015/level1").filter(ee.Filter.eq('ADM1_NAME','Canterbury'))
sentinel2 = geemap.ee.ImageCollection('COPERNICUS/S2_SR')

start_date = '2022-01-01'
end_date = '2022-12-31'

filtered = sentinel2.filterBounds(roi.geometry()).filterDate(start_date, end_date)
def calculate_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

ndvi_collection = filtered.map(calculate_ndvi)

chart = geemap.chart(ndvi_collection, region=roi.geometry(), reducer=ee.Reducer.mean(), scale=10)
chart.style(width='800px', height='300px', position='bottomright')

Map.add_child(chart)

Map.centerObject(roi.geometry(), 10)
Map.addLayer(ndvi_collection.median(), {'min': 0, 'max': 1, 'palette': ['red', 'yellow', 'green']}, 'NDVI')

Map


AttributeError: module 'geemap' has no attribute 'chart'